In [1]:
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime

from tensorflow import keras
import tensorflow as tf
import numpy as np

from OperatorLayer import SymmetricOperator
from NormalizedMeanSquaredError import NormalizedMeanSquaredError

In [2]:
data_name = 'Duffing_Equation_expt4'  ## FILL IN HERE (from file name)
data_folder = '../NODE-Operators/data/'

# data is num_steps x num_examples x n
data_train_u = np.load(data_folder + "{}_train1_u.npy".format(data_name))
data_train_f = np.load(data_folder + "{}_train1_f.npy".format(data_name))

# data is num_steps x num_examples x n
data_val_u = np.load(data_folder + "{}_val_u.npy".format(data_name))
data_val_f = np.load(data_folder + "{}_val_f.npy".format(data_name))

# data is num_steps x num_examples x n
data_test_u = np.load(data_folder + "{}_test2_u.npy".format(data_name))
data_test_f = np.load(data_folder + "{}_test2_f.npy".format(data_name))

In [3]:
# %load_ext tensorboard

def construct_encoder(n, l, act_layer_config, lin_layer_config, name):
    # First build the encoder
    input_ = keras.layers.Input(shape=n)
    hidden1 = keras.layers.Dense(n, **act_layer_config)(input_)
    hidden2 = keras.layers.Dense(n, **act_layer_config)(hidden1)
    hidden3 = keras.layers.Dense(n, **lin_layer_config)(hidden2)
    added = keras.layers.Add()([input_, hidden3])
    latentspace = keras.layers.Dense(l, **lin_layer_config)(added)
    encoder = keras.Model(inputs=[input_], outputs=[latentspace], name=name)
    return encoder

def construct_decoder(n, l, act_layer_config, lin_layer_config, name):
    # Now the decoder
    latent_ = keras.layers.Input(shape=l)
    hidden4 = keras.layers.Dense(n, **lin_layer_config)(latent_)
    hidden5 = keras.layers.Dense(n, **act_layer_config)(hidden4)
    hidden6 = keras.layers.Dense(n, **act_layer_config)(hidden5)
    hidden7 = keras.layers.Dense(n, **act_layer_config)(hidden6)
    added_ = keras.layers.Add()([hidden4, hidden7])
    decoder = keras.Model(inputs=[latent_], outputs=[added_], name=name)
    return decoder


In [4]:
_, n = data_train_u.shape
l = 20

print("Training contains:", data_train_u.shape[0], "samples.")
print("Validation contains:", data_val_u.shape[0], "samples.")
print("Input vector is", n, "neurons and latent space is", l, "neurons.")

Training contains: 4798 samples.
Validation contains: 1200 samples.
Input vector is 128 neurons and latent space is 20 neurons.


In [5]:
# Set the configuration to be used for layers with activation functions and linear, non-activated functions
#act_layer = dict(activation="relu", kernel_initializer='he_normal')
act_layer = dict(activation="elu", kernel_initializer='he_normal')
lin_layer = dict(activation=None)

# Encoder and decoder for u
u_enc = construct_encoder(n, l, act_layer, lin_layer, "u_encoder")
u_dec = construct_decoder(n, l, act_layer, lin_layer, "u_decoder")

# Encoder and decoder for u
f_enc = construct_encoder(n, l, act_layer, lin_layer, "f_encoder")
f_dec = construct_decoder(n, l, act_layer, lin_layer, "f_decoder")

In [6]:
# Set the optimizer to be used
optimizer = keras.optimizers.SGD(lr=0.01)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

# Specify fit options
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

cbs = [keras.callbacks.ModelCheckpoint("dae.tf", save_best_only=True),
       keras.callbacks.EarlyStopping(),
       keras.callbacks.TensorBoard(log_dir=logdir)]

fit_options = dict(batch_size = 20, epochs = 5)

In [7]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
encoder_loss = NormalizedMeanSquaredError()

In [8]:
# Now assemble the COMPLETE, LINKED autoencoder!!
u_input = keras.layers.Input(shape=n)
u_encoded = u_enc(u_input)

f_input = keras.layers.Input(shape=n)
f_encoded = f_enc(f_input)

#Operator = SymmetricOperator()
#L = Operator.get_operator()
#OperatorLayer = Operator(u_encoded)

L_full = tf.Variable(tf.eye(l), trainable=True, dtype=tf.float32, name="L_full")
L_upper = tf.linalg.band_part(L_full, 0, -1, name="L_upper")
L = tf.multiply(0.5,L_upper+tf.transpose(L_upper), name="L")
OperatorLayer = tf.matmul(u_encoded, L, name="OperatorLayer")

f_encoded_T = tf.transpose(f_encoded)
Linvf_T = tf.linalg.solve(L, f_encoded_T, adjoint=True)
Linvf = tf.transpose(Linvf_T)

#DiffLayer = keras.layers.Subtract()([OperatorLayer, f_encoded])

u_decoded = u_dec(u_encoded)
f_decoded = f_dec(f_encoded)

Lv_decoded = f_dec(OperatorLayer)
Linvf_decoded = u_dec(Linvf)

linked_aec = keras.Model(inputs = [u_input, f_input], 
                         outputs = [u_decoded, f_decoded, Lv_decoded, Linvf_decoded], name="linked_autoencoders")

f_sums = tf.reshape(f_encoded[None]+f_encoded[:,None], [-1,l])
Lv_sums = tf.reshape(OperatorLayer[None]+OperatorLayer[:,None], [-1,l])

linked_aec.add_loss(encoder_loss(f_encoded, OperatorLayer))
linked_aec.add_loss(encoder_loss(f_sums, Lv_sums))

In [9]:
u_enc.summary()
u_dec.summary()
f_enc.summary()
f_dec.summary()
linked_aec.summary()


Model: "u_encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          16512       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________

In [10]:
#linked_aec.compile(loss=["mse","mse", "mse"], optimizer=optimizer)
linked_aec.compile(loss=[encoder_loss, encoder_loss, encoder_loss, encoder_loss], optimizer=optimizer)

In [11]:
%%time
train_zeros = np.zeros((data_train_u.shape[0], l))
val_zeros = np.zeros((data_val_u.shape[0], l))

val_data = [(data_val_u, data_val_f), 
            (data_val_u, data_val_f, data_val_f, data_val_u)]

hist = linked_aec.fit(x=[data_train_u, data_train_f], 
                      y=[data_train_u, data_train_f, data_train_f, data_train_u], 
                      validation_data=val_data,
                      callbacks=cbs,
                      **fit_options)

Train on 4798 samples, validate on 1200 samples
Epoch 1/5
4780/4798 [============================>.] - ETA: 0s - loss: 0.6981 - u_decoder_loss: 0.0558 - f_decoder_loss: 0.1868 - f_decoder_1_loss: 0.1920 - u_decoder_1_loss: 0.0950WARNING:tensorflow:From /anaconda3/envs/tensorflow3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: dae.tf/assets
4798/4798 [==============================] - 47s 10ms/sample - loss: 0.6969 - u_decoder_loss: 0.0557 - f_decoder_loss: 0.1865 - f_decoder_1_loss: 0.1917 - u_decoder_1_loss: 0.0949 - val_loss: 0.2876 - val_u_decoder_loss: 0.0261 - val_f_decoder_loss: 0.0863 - val_f_decoder_1_loss: 0.0910 - val_u_decoder_1_loss: 0.0376
Epoch 2/5
4798/4798 [===

In [12]:
#encoder_loss = NormalizedMeanSquaredError()
#linked_aec = keras.models.load_model("dae.tf", custom_objects={"NormalizedMeanSquaredError": NormalizedMeanSquaredError})
#linked_aec = keras.models.load_model("dae.tf")

In [13]:
u_aec, f_aec, f_pred, u_pred = linked_aec.predict(x=[data_train_u, data_train_f])

In [14]:
tf.shape(u_pred)

<tf.Tensor: id=40638, shape=(2,), dtype=int32, numpy=array([4798,  128], dtype=int32)>

In [15]:
ind = 0
u_exact = data_train_u[ind,:]
u_network = u_pred[ind,:]

In [16]:
#x_pts = np.linspace(0,2*np.pi,n)
#plt.figure() 
#plt.plot(x_pts,u_exact,'C1',linewidth=2,label="u exact")
#plt.plot(x_pts,u_network,'o--k',markersize=5,label="u network")
#plt.legend(loc="upper right",fontsize=16)
#plt.xticks(fontsize=16)
#plt.yticks(fontsize=16)
#plt.tight_layout()
#plt.show()